In [1]:
import geemap
import ee
ee.Initialize(project='pyregence-ee')
import scripts.analysis_functions as af
import scripts.fire_info as fi
import scripts.get_image_collections as gic
import scripts.utils as utils

In [11]:
def bs_calc(feat) -> ee.ImageCollection:
    
    feature = ee.Feature(feat)
    region, fire_start, fire_end, pre_start, pre_end = fi.get_fire_info_from_feature(feature) 
    
    # landsatCol : None, t2, bestls
    # None=T1 real time toa, t2=None+ T1 real time toa, bestls=T1 toa
    # cloudBustingMethod : None, bust
    # None=No cloud masking/busting, bust=cloud busting
    # sensor : landsat, sentinel2
    sensor = "landsat"
    
    pre_collection = gic.get_image_collection(sensor,region,pre_start,pre_end,landsatCol='bestls',cloudBustingMethod='bust')
    pre_img = gic.get_composite(pre_collection,gic.make_pre_composite,pre_start,pre_end)
  
    post_collection = gic.get_image_collection(sensor,region,fire_start,fire_end,landsatCol='bestls', cloudBustingMethod='bust')
    post_img = gic.get_composite(post_collection,gic.make_pre_composite,fire_start, fire_end)

    rdnbr = af.rdnbr(pre_img,post_img)
    miller = af.miller_thresholds(rdnbr)

    return ee.Image(miller).clip(region).select('MillersThresholds').toByte()

# version 1 uses 1yr prior / 1 yr after for pre-fire and post-fire assessment taking 3mo composites straddling fire discovery date
# featCollv1 = ee.FeatureCollection('projects/pyregence-ee/assets/conus/nifc/nifc_fires_2020_gte500acres_v1')
# print(f'computing burn severity for {featCollv1.size().getInfo()} fires')
# bs_collv1 = ee.FeatureCollection(featCollv1).map(bs_calc)
# bs_finalv1 = ee.ImageCollection(bs_collv1).max().rename('SEVERITY')

# # version 2 uses same yr / 1yr after for pre-fire and post-fire assessment, taking 3mo composite leading up to fire discovery date
# featCollv2 = ee.FeatureCollection('projects/pyregence-ee/assets/conus/nifc/nifc_fires_2020_gte500acres_v2')
# print(f'computing burn severity for {featCollv2.size().getInfo()} fires')
# bs_collv2 = ee.FeatureCollection(featCollv2).map(bs_calc)
# bs_finalv2 = ee.ImageCollection(bs_collv2).max().rename('SEVERITY')

# version 3 uses 1yr before / 1yr after for pre-fire and post-fire assessment, taking 3mo composite leading up to fire discovery date
featCollv3 = ee.FeatureCollection('projects/pyregence-ee/assets/conus/nifc/nifc_fires_2020_gte500acres_v3')
print(f'computing burn severity for {featCollv3.size().getInfo()} fires')
bs_collv3 = ee.FeatureCollection(featCollv3).map(bs_calc)
bs_finalv3 = ee.ImageCollection(bs_collv3).max().rename('SEVERITY')

featColl = ee.FeatureCollection("projects/pyregence-ee/assets/conus/nifc/calfire_frap_fires_2020_gte500acres")
print(f'computing burn severity for {featColl.size().getInfo()} fires')
bs_coll = ee.FeatureCollection(featColl).map(bs_calc)
bs_final = ee.ImageCollection(bs_coll).max().rename('SEVERITY')

bs_nifc_plus_calfire = ee.ImageCollection([bs_finalv3,bs_final]).max()

computing burn severity for 478 fires
computing burn severity for 93 fires


In [12]:
Map = geemap.Map()
Map.addLayer(bs_finalv3, {"min":0,"max":3,"palette":["00ff1f","fbff0e","ffbc00","ff0000"]},'nifc bs v3', False)
Map.addLayer(bs_final, {"min":0,"max":3,"palette":["00ff1f","fbff0e","ffbc00","ff0000"]},'calfire bs', False)
Map.addLayer(bs_nifc_plus_calfire, {"min":0,"max":3,"palette":["00ff1f","fbff0e","ffbc00","ff0000"]},'calfire + nifc bs')

# Map.addLayer(bs_finalv1, {"min":0,"max":3,"palette":["00ff1f","fbff0e","ffbc00","ff0000"]},'bs v1')
# Map.addLayer(bs_finalv2, {"min":0,"max":3,"palette":["00ff1f","fbff0e","ffbc00","ff0000"]},'bs v2')
# Map.addLayer(bs_finalv3, {"min":0,"max":3,"palette":["00ff1f","fbff0e","ffbc00","ff0000"]},'bs v3')

#Map.addLayer(featCollv1, {'color':'pink'}, 'fires', True,0.4)
Map.setCenter(-112, 39, 6)

Map

Map(center=[39, -112], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [13]:
# export img to asset
# desc = 'bs_2020fires_gte500ac_v1'
# utils.exportImgtoAsset(bs_finalv1, desc=desc, export=True)

# desc = 'bs_2020fires_gte500ac_v2'
# utils.exportImgtoAsset(bs_finalv2, desc=desc, export=True)

# desc = 'bs_2020fires_gte500ac_v3'
# utils.exportImgtoAsset(bs_finalv3, desc=desc, export=True)

desc = 'bs_2020fires_gte500ac_2021_11_16'
utils.exportImgtoAsset(bs_nifc_plus_calfire, desc=desc, export=True)

export task started
